# QR factorization for text retrieval
In this notebook, we would like to compare the cosine similarity with the QR factorization in the context of text retrieval. So, given the term-document matrix A and the two query vectors defined in the following code, we compute the cosine similarity between the query vectors and the matrix A to find which documents are meaningful for the given query. Then, we define an additional query and analyze the result. 

## Import libraries

In [1]:
import numpy as np
import scipy.linalg as spl
import sklearn.feature_extraction

## Text retrieval with cosine similarity

Define the term-by-document matrix

In [2]:
np.set_printoptions(suppress=True)
vectorizer = sklearn.feature_extraction.text.CountVectorizer(min_df=1)
documents = [
'The rank of a matrix is the maximum number of linearly independent columns.',
'The column space of a matrix A is called range of A.',
'The two norm of a vector is called Euclidean norm.',
'The function norm satisfies the triangular inequality.',
'The inverse of an orthogonal matrix is its transpose.',
'The product of an orthogonal matrix and its traspose is the identity matrix.',
'The matrix vector product is not commutative',
'The rank of a matrix is the maximum number of  linearly independent rows.',
'A set of orthogonal vectors is a linearly independent set.',
'The columns of an orthogonal matrix are a set of orthogonal vectors.',
'A normed vectorial space is a space with an inner product norm.',
'An inner product space is a vectorial space with an inner product',
'A norm can be induced by an inner product.']

In [3]:
X = vectorizer.fit_transform(documents).toarray()
print('vectorized.vocabulary_: {0}'.format(vectorizer.vocabulary_))
A=X.T # we compute the transpose and work with this to have on the rows the terms, and on the columns the documents

vectorized.vocabulary_: {'the': 36, 'rank': 31, 'of': 27, 'matrix': 21, 'is': 18, 'maximum': 22, 'number': 26, 'linearly': 20, 'independent': 13, 'columns': 8, 'column': 7, 'space': 35, 'called': 5, 'range': 30, 'two': 40, 'norm': 23, 'vector': 41, 'euclidean': 10, 'function': 11, 'satisfies': 33, 'triangular': 39, 'inequality': 15, 'inverse': 17, 'an': 0, 'orthogonal': 28, 'its': 19, 'transpose': 37, 'product': 29, 'and': 1, 'traspose': 38, 'identity': 12, 'not': 25, 'commutative': 9, 'rows': 32, 'set': 34, 'vectors': 43, 'are': 2, 'normed': 24, 'vectorial': 42, 'with': 44, 'inner': 16, 'can': 6, 'be': 3, 'induced': 14, 'by': 4}


In [4]:
(m,n)=A.shape
print(f'The shape of the term-by-document matrix is ({m},{n})')
print(f'There are {m} terms and {n} documents.')

The shape of the term-by-document matrix is (45,13)
There are 45 terms and 13 documents.


Now we want to compute the norm of each column of our matrix A, and then we want to divide each column by its norm, to have at the end each column with norm = 1. In this way, we can simply the computation of the cosine since we don't need to divide by the norm of each column of the matrix (because it is already 1).

In [5]:
eu=np.array(np.zeros(n))
for i in range(n):
    eu[i] = np.linalg.norm(A[:,i],2)
As = np.dot(A,np.diag(1/eu))

Now we can define the first query vector.

In [6]:
query1text = ['rank of a matrix?']
query1 = vectorizer.transform(query1text).toarray()
print('query-vector: \n{0}'.format(query1))
print('The shape of the query-vector:',query1.shape)
query1 = query1.T
print('The shape of the query-vector after transposition:',query1.shape)

query-vector: 
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 0]]
The shape of the query-vector: (1, 45)
The shape of the query-vector after transposition: (45, 1)


Now we would like to compute the cosine of the angle between the query one and each column of the term-by-document matrix. In general, 

$cos(\theta) = \frac{x^Ty}{\|x\|\|y\|}$. If we have a term-by-document matrix, $cos(\theta_j) = \frac{A_{*j}q}{\|A_{*j}\|\|q\|}$. We can simply compute the dot product between the matrix and the query to compute one-shot all the cosine. <br>
<b>NB</b>: in this case, we don't divide by the norm of the matrix because we have already divided each column by its own norm and so the norm of the matrix is equal one.<br> At the end, fixed a threshold equal to 0.5, we can retrieve the document releated to the defined query. Usually, less than 0.5 is not good, and so we don't take anything.

In [7]:
cos_query1 = np.dot(query1.T,As)/np.linalg.norm(query1,2)
print('The cosine similarity releated to the query1 are:\n', cos_query1)
print('Query1 is: ', query1text)
print('Founded documents are:\n',
      [documents[i] for i in np.where(cos_query1 == np.max(cos_query1))[1]])

The cosine similarity releated to the query1 are:
 [[0.57735027 0.52223297 0.17407766 0.         0.38490018 0.42008403
  0.21821789 0.57735027 0.18257419 0.4472136  0.         0.
  0.        ]]
Query1 is:  ['rank of a matrix?']
Founded documents are:
 ['The rank of a matrix is the maximum number of linearly independent columns.', 'The rank of a matrix is the maximum number of  linearly independent rows.']


Now we define a second query vector.

In [8]:
query2text = ['euclidean norm']
query2 = vectorizer.transform(query2text).toarray()
print('query-vector: \n{0}'.format(query2))
print('The shape of the query-vector:',query2.shape)
query2 = query2.T
print('The shape of the query-vector after transposition:',query2.shape)
# scaled query vector
#query2=query2/np.linalg.norm(query2,2)

query-vector: 
[[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0]]
The shape of the query-vector: (1, 45)
The shape of the query-vector after transposition: (45, 1)


In [9]:
cos_query2 = np.dot(query2.T,As)/np.linalg.norm(query2,2)
print('The cosine similarity releated to the query2 are:\n', cos_query2)
print('Query2 is: ', query2text)
print('Founded documents are:\n',
      [documents[i] for i in np.where(cos_query2 == np.max(cos_query2))[1]])

The cosine similarity releated to the query2 are:
 [[0.         0.         0.63960215 0.23570226 0.         0.
  0.         0.         0.         0.         0.20412415 0.
  0.25      ]]
Query2 is:  ['euclidean norm']
Founded documents are:
 ['The two norm of a vector is called Euclidean norm.']


## Text retrieval using QR factorization with pivoting
Compute the QR factorization, with pivoting true, of the scaled term-document matrix and use the QR factors to compute the cosine similarity. Compute an approximation of the term-document matrix by using nc columns of Q and nc rows of R (as in the first exercise). Use this approximation to compute again the cosine similarity and comment the results.

Computing the QR with pivoting, we can check that all the diagonal elements of the matrix R are sorted. In general, it is necessary to use column pivoting during the QR factorization to ensure that the zeros appear at the bottom of the matrix AP = QR.

In [10]:
[Q,R,P]=spl.qr(As,pivoting=True)
print(np.diag(R))

[-1.         -1.         -0.93541435  0.90965028 -0.90475592 -0.89621574
  0.8327188  -0.82051483  0.70238715 -0.65504204 -0.56200771 -0.41935821
  0.32819427]


Now we put r = 10, and we take from the matrix Q just the first r columns, and from the matrix R the first r rows.

In [11]:
r = 8

So, we can compute the matrix $Q_A = Q_{*,0..r}$ which defines the basis for the column space—R(A). The columns of QAO are a basis for the orthogonal complement of the column space of AP and so of the column space of A. We do the same for $R_A = Q_{0..r,*}$.
Column pivoting provides important numerical advantages without changing the database, as permuting the columns of A results only in a reordering of the document vectors.

In [12]:
QA=np.copy(Q[:,0:r])
QAO=np.copy(Q[:,r:m])
RA=np.copy(R[0:r,:])

In [13]:
costet=np.dot(As[:,P].T,query1)/np.linalg.norm(query1,2)
print('costeta between matrix and query 1:\n',np.transpose(costet))

costet=np.dot(RA.T,np.dot(QA.T,query1))/np.linalg.norm(query1,2)
print('costeta between the approximated matrix and query 1:\n',np.transpose(costet))

costeta between matrix and query 1:
 [[0.57735027 0.         0.         0.         0.18257419 0.21821789
  0.17407766 0.38490018 0.52223297 0.4472136  0.42008403 0.
  0.57735027]]
costeta between the approximated matrix and query 1:
 [[ 0.57735027  0.         -0.         -0.          0.18257419  0.21821789
   0.17407766  0.38490018  0.36863684  0.31610915  0.35157499  0.02968657
   0.53878653]]


In [14]:
RAs=np.copy(R) 
RAs[r:m,:]=0
E = np.dot(Q,RAs)-np.dot(Q,R)
AE = As+E
AEeu=np.array(np.zeros((1,n)))
for i in range(n):
    AEeu[0,i] = np.linalg.norm(AE[:,i],2)

QAs=np.copy(Q[:,0:r])
# QAs basis for the column space of a subsapce of the columns space A
RAs=RAs[0:r,:]

#(QAs,RAs)=np.linalg.qr(AE)

costeta1=np.dot(RAs.T,np.dot(QAs.T,query1))/(AEeu.T * np.linalg.norm(query1, 2))
print('cos query 1')
print(np.transpose(costeta1))

print('cos query 2')
costeta2=np.dot(RAs.T,np.dot(QAs.T,query2))/(AEeu.T * np.linalg.norm(query2, 2))
print(np.transpose(costeta2))

cos query 1
[[ 0.57735027  0.         -0.         -0.          0.18257419  0.21821789
   0.17407766  0.38490018  0.30166037  0.42352088  0.30319247  0.03277066
   0.50950867]]
cos query 2
[[ 0.          0.25        0.23570226  0.20412415  0.          0.
   0.63960215  0.          0.11278966 -0.00045259 -0.00840464  0.05663592
   0.0103117 ]]
